# MLM Training

In this notebook we'll cover the training process for masked-language modeling (MLM). First we import and initialize everything required.

In [0]:
# PARTICIONO EL TEXTO X PUNTOS Y QTY DE PALABRAS
# APLICO MISMO POST-PROCESING HECHO PARA EL CORPUS DE BETO
# POST-PROCESING PROPIO DEL CORPUS PROPIO (EJEMPLO, PARA CHAT SOLO DEJAR LO QUE SIGUE A "CLIENTE:")
# APICAR AUTOCORRECTOR
#IDENTIFICAR SI HAY NUEVOS TOKENS (NO HAY TANTOS LA VERDAD, PORQUE SON SEGMENTOS DE PALABRAS, NO PALABRAS)

In [0]:
%pip install transformers==4.4
%pip install torch==1.9.0
%pip install spacy==3.1.3

Python interpreter will be restarted.
Requirement already satisfied: transformers==4.4 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-9f469ca8-2890-4e78-80d4-138275a2b980/lib/python3.8/site-packages (4.4.0)
Requirement already satisfied: sacremoses in /local_disk0/.ephemeral_nfs/envs/pythonEnv-9f469ca8-2890-4e78-80d4-138275a2b980/lib/python3.8/site-packages (from transformers==4.4) (0.0.53)
Requirement already satisfied: tqdm>=4.27 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-9f469ca8-2890-4e78-80d4-138275a2b980/lib/python3.8/site-packages (from transformers==4.4) (4.64.0)
Requirement already satisfied: numpy>=1.17 in /databricks/python3/lib/python3.8/site-packages (from transformers==4.4) (1.19.2)
Requirement already satisfied: regex!=2019.12.17 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-9f469ca8-2890-4e78-80d4-138275a2b980/lib/python3.8/site-packages (from transformers==4.4) (2022.6.2)
Requirement already satisfied: filelock in /usr/local/lib/python3.8/dist-packages (from transformers==4.4) (3.0.12)
Requirement already satisfied: tokenizers<0.11,>=0.10.1 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-9f469ca8-2890-4e78-80d4-138275a2b980/lib/python3.8/site-packages (from transformers==4.4) (0.10.3)
Requirement already satisfied: requests in /databricks/python3/lib/python3.8/site-packages (from transformers==4.4) (2.25.1)
Requirement already satisfied: packaging in /databricks/python3/lib/python3.8/site-packages (from transformers==4.4) (20.9)
Requirement already satisfied: pyparsing>=2.0.2 in /databricks/python3/lib/python3.8/site-packages (from packaging->transformers==4.4) (2.4.7)
Requirement already satisfied: certifi>=2017.4.17 in /databricks/python3/lib/python3.8/site-packages (from requests->transformers==4.4) (2020.12.5)
Requirement already satisfied: chardet<5,>=3.0.2 in /databricks/python3/lib/python3.8/site-packages (from requests->transformers==4.4) (4.0.0)
Requirement already satisfied: urllib3<1.27,>=1.21.1 in /databricks/python3/lib/python3.8/site-packages (from requests->transformers==4.4) (1.25.11)
Requirement already satisfied: idna<3,>=2.5 in /databricks/python3/lib/python3.8/site-packages (from requests->transformers==4.4) (2.10)
Requirement already satisfied: six in /databricks/python3/lib/python3.8/site-packages (from sacremoses->transformers==4.4) (1.15.0)
Requirement already satisfied: joblib in /databricks/python3/lib/python3.8/site-packages (from sacremoses->transformers==4.4) (1.0.1)
Requirement already satisfied: click in /local_disk0/.ephemeral_nfs/envs/pythonEnv-9f469ca8-2890-4e78-80d4-138275a2b980/lib/python3.8/site-packages (from sacremoses->transformers==4.4) (8.1.3)
WARNING: You are using pip version 21.0.1; however, version 22.1.2 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-9f469ca8-2890-4e78-80d4-138275a2b980/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.
Python interpreter will be restarted.
Requirement already satisfied: torch==1.9.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-9f469ca8-2890-4e78-80d4-138275a2b980/lib/python3.8/site-packages (1.9.0)
Requirement already satisfied: typing-extensions in /local_disk0/.ephemeral_nfs/envs/pythonEnv-9f469ca8-2890-4e78-80d4-138275a2b980/lib/python3.8/site-packages (from torch==1.9.0) (4.2.0)
WARNING: You are using pip version 21.0.1; however, version 22.1.2 is available.
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-9f469ca8-2890-4e78-80d4-138275a2b980/bin/python -m pip install --upgrade pip' command.
Python interpreter will be restarted.
Python interpreter will be restarted.
Requirement already satisfied: spacy==3.1.3 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-9f469ca8-2890-4e78-80d4-138275a2b980/lib/python3.8/site-packages (3.1.3)
Requirement already satisfied: murmurhash<1.1.0,>=0.28.0 in /local_disk0/.ephemeral_nfs/envs/pythonEnv-9f469ca8-2890-4e78-80d4-138275a2b980/lib/python3.8/site-packages (from spacy==3.1.

In [0]:
# https://www.youtube.com/watch?v=R6hcxMMOrPE
# https://towardsdatascience.com/how-to-train-bert-aaad00533168

from transformers import BertTokenizer, BertForMaskedLM
import torch
import pandas as pd
import re

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-uncased")
# model = BertForMaskedLM.from_pretrained('bert-base-uncased')
model = BertForMaskedLM.from_pretrained("/dbfs/Efecty_VoC/BETOmodel/BETO/")

Some weights of the model checkpoint at /dbfs/Efecty_VoC/BETOmodel/BETO/ were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).

We'll be using *Meditations* by *Marcus Aurelius* as our training data. The file below has already been cleaned and so no further processing is required (beyond the `split`).

In [0]:
pd.set_option('display.max_colwidth',-1)

<command-1401231740186972>:1: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
 pd.set_option('display.max_colwidth',-1)

In [0]:
import unicodedata
import sys

def _is_punctuation(char):
  """Checks whether `chars` is a punctuation character."""
  cp = ord(char)
  # We treat all non-letter/number ASCII as punctuation.
  # Characters such as "^", "$", and "`" are not in the Unicode
  # Punctuation class but we treat them as punctuation anyways, for
  # consistency.
  if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
      (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
    return True
  cat = unicodedata.category(char)
  if cat.startswith("P"):
    return True
  return False

def _run_split_on_punc(text):
    """Splits punctuation on a piece of text."""
    chars = list(text)
    i = 0
    start_new_word = True
    output = []
    while i < len(chars):
      char = chars[i]
      if _is_punctuation(char):
        output.append([char])
        start_new_word = True
      else:
        if start_new_word:
          output.append([])
        start_new_word = False
        output[-1].append(char)
      i += 1

    return ' '.join(["".join(x) for x in output])

#Eliminar multiespacios en blanco
def replace_multi_whitespaces(line):
	return ' '.join(line.split())


In [0]:
df = pd.read_csv(f"/dbfs/Efecty_VoC/BETOmodel/Datasets/Tema - Colab Encuesta.csv", sep=',')
df.columns=['text','otra']

df['text']=df['text'].apply(lambda x: x.lower())

for t in range(0,len(df['text'])):
     df['text'][t]=replace_multi_whitespaces(_run_split_on_punc(df['text'][t]))

In [0]:
####POST PROCESING DEL CORPUS QUE HACEN EN BETO

# corpus_processing.py
# import re
# import sys


#URLS_RE = re.compile(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b')
# URLS_RE = re.compile(r'((http|https)\:\/\/)?[a-zA-Z0-9\.\/\?\:@\-_=#]+\.([a-zA-Z]){2,6}([a-zA-Z0-9\.\&\/\?\:@\-_=#])*')

# LISTING_RE = re.compile(r'^(|[a-z]?|[0-9]{0,3})(\-|\.)+( |\n)')

# def remove_urls(text):
# 	return URLS_RE.sub('', text)

# def replace_multi_whitespaces(line):
# 	return ' '.join(line.split())

# def remove_listing(line):
# 	return LISTING_RE.sub('', line)

# def main():	

# 	with open(sys.argv[1], "r") as input_file:

# 		for line in input_file:
# 			if line is '\n':
# 				print('')
# 			else:
# 				line = line.lower()
# 				line = remove_urls(line)
# 				line = remove_listing(line)
# 				line = replace_multi_whitespaces(line)

# 				if line is not '':
# 					print(line)



# if __name__ == '__main__':
#     main()
    
    
   

In [0]:
# Tratamiento de datos
index_ini=list(df.index)
df.loc[index_ini,'text']=df['text'].apply(lambda x: re.sub(r'[\n]{1,4}',' ',x)) #Mas que nada para las PQRS que tienen muchas lineas vacias, donde hay mucho \n textual
df.loc[index_ini,'text']=df['text'].apply(lambda x: re.sub(r"""[' ']{1,4}""",' ',x)) 

df.loc[index_ini,'text']=df['text'].apply(lambda x: re.findall(".liente:(.*)\**",x) if 'liente:' in x else x)
df.loc[index_ini,'text']=df['text'].apply(lambda x: re.sub(r'\?','zzz',str(x)))
df.loc[index_ini,'text']=df['text'].apply(lambda x: re.sub(r'\.','bbb',str(x)))
df.loc[index_ini,'text']=df['text'].apply(lambda x: re.sub(',','qwqz',str(x)))
df.loc[index_ini,'text']=df['text'].apply(lambda x: re.sub('[\W]',' ',str(x)))
df.loc[index_ini,'text']=df['text'].apply(lambda x: re.sub('qwqz',',',str(x)))
df.loc[index_ini,'text']=df['text'].apply(lambda x: re.sub('bbb','.',str(x)))
df.loc[index_ini,'text']=df['text'].apply(lambda x: re.sub('zzz','?',str(x)))
df.loc[index_ini,'text']=df['text'].apply(lambda x: re.sub(r'\s\s',' ',str(x)))
df.loc[index_ini,'text']=df['text'].apply(lambda x: re.sub('Imagen .*? [0-9-a-z]{10,30}','',str(x)))

In [0]:
#TRABAJAR SOBRE EL CORRECTOR 

# from spellchecker import SpellChecker
# spanish = SpellChecker(language='es') 

# text_df=pd.DataFrame(text)
# text_df[0]=text_df[0].apply(lambda x: x.lower())
# text_df[1]='0'

# for i in range(0,len(text_df[0])):
#     s_split= text_df[0][i].split(' ')
#     prueba=spanish.unknown(s_split)
#     for word in prueba:
#       text_df[1][i]=spanish.correction(word)

# text_df[0]=text_df[0].apply(lambda x: x.lower())

# pd.set_option('display.max_colwidth', -1)
# text_df[text_df[1]!='0']

In [0]:
#UPDATE TOKENS 

#Generar una lista de los tokens que bert considera unknow. Para beto es el token 3
# add_tokens=[]
# for j in range(0,len(text)):
#    word_t=text[j].split(' ')
#    for i in word_t:
#      if tokenizer.convert_tokens_to_ids(tokenizer.tokenize(str(word_t)))==[3]:
#        add_tokens.append(word_t)



ej: tokenizer.convert_tokens_to_ids(tokenizer.tokenize("😁"))==[3]
# special tokens: "[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]
# luego,
# tokenizer.add_tokens(['new', 'rdemorais', 'blabla']) #Identificar los tokens que no estan en tokenizer (token=100)
# model.resize_token_embeddings(len(tokenizer)) # el shape de los embeddings depende de la qty de palabras, por eso hay que hacer un reshape

In [0]:
tokenizer.convert_tokens_to_ids(tokenizer.tokenize("[MASK]"))
# cls=4,sep=5,pad=1
# mask=0

Out[9]: [0]

In [0]:
text=df['text'].tolist()
text[100:110]

Out[10]: ['en los proyectos para pagos como revistas o claro o movistar deben indicar el nombre completo del titular para estar mas segura del pago . el tema de trabajar los sábados mínimo 10 horas y domingos mínimo 5 horas , no va conmigo para mi es mucho mas importante dedicarle tiempo a mi esposo e hijas , no todo en la vida es trabajo . y por ultimo en mi punto como tema personal me parece injusto que no pueda recaudar otras facturas que porque efecty no tiene el convenio , de 5 años que llevo como independiente siempre me he caracterizado por la excelente atención y solución que le doy al cliente . así no tenga convenio busco la manera de realizar los pagos así sea que me toque ir o mandar a alguien a pagarlos en otro lado . ejemplo electrohuila . deben ser mas asequibles y no ser egoístas en pensar solo en ustedes como marca , la idea es darle solución a los clientes que se vayan satisfechos porque encuentran lo que realmente necesitan en un punto de atención . eso permite que el voz a voz se multiplique y sea más la gente que va a llegar no solo a pagar facturas si no también a realizar giros y demás cosas . gracias ',
 'en la mayoria de proyectos se pueda ver nombre del titular ejemplo pagos de cuotas banco , movil , tv para informarle al cliente el pago muchas veces envian otra persona a pagar',
 'la mayor in5 es la disminución de las contraprestaciones , mientras el aumento del dinero que se maneja en el punto ha sido el doble . más riesgo para uno y menos beneficio',
 'tener en cuenta que los puntos de atención deben de tener una distancia entre uno y otro en mi caso este punto tiene mas de 7 años funcionando en el mismo local y la señora de la esquina coloco el que ella tenia en otra dirección a una cuadra de este , lo que es competencia desleal . gracias',
 'es importante que los logísticos tengan la información de los convenios nuevos que salen en especial los bingos , los cambios de código de las empresas y en especial se pueda tener una mejor comunicación ya sea por celular o watsapp , por que en eso estamos fallando mucho .',
 'tratar de no perder algunos convenios con las empresas , ya que los clientes se sienten muy satisfechos con el servicios por que practicamente hacen todos sus pagos en un solo lugar .',
 'buenas tardes que por favor no mas aperturas de efecty esto no da para tanto la situacion no es muy buena , los bancos con esas plataformas de nequi y daviplata nos llevan ventaja no cobran por los giros y eso a bajado el envio y retiro de los giros .',
 'ampliar mas pap para brindar servicios a los usuarios en todo el territorio nacional .',
 'si es posible , habilitar recaudos daviplata',
 'hacer mas convenios con 4 banco , los cuales están brindados servicios que las nuevas generaciones están utilizando mucho']

First, we'll tokenize our text.

In [0]:
inputs = tokenizer(text, return_tensors='pt', max_length=512, truncation=True, padding='max_length')

Then we create our *labels* tensor by cloning the *input_ids* tensor.

In [0]:
inputs['labels'] = inputs.input_ids.detach().clone()

In [0]:
inputs.keys()

Out[13]: dict_keys(['input_ids', 'token_type_ids', 'attention_mask', 'labels'])

Now we mask tokens in the *input_ids* tensor, using the 15% probability we used before - and the **not** a *CLS* or *SEP* token condition. This time, because we have padding tokens we also need to exclude *PAD* tokens (*0* input ids).

In [0]:
# create random array of floats with equal dimensions to input_ids tensor
rand = torch.rand(inputs.input_ids.shape)
# create mask array
mask_arr = (rand < 0.15) * (inputs.input_ids != 4) * \
           (inputs.input_ids != 5) * (inputs.input_ids != 1)

And now we take take the indices of each `True` value, within each individual vector.

In [0]:
selection = []
qty_train=inputs.input_ids.shape[0]

for i in range(qty_train):
    selection.append(
        torch.flatten(mask_arr[i].nonzero()).tolist() #identifica el index que va a representar el 15% mask para los inputs quienes seran los outputs. 77 numeros (15% de 512)        
    )

Then apply these indices to each respective row in *input_ids*, assigning each of the values at these indices as *103*.

In [0]:
# Al 15% que se identifico de forma random para testear, en el input va a estar como 103 
for i in range(inputs.input_ids.shape[0]):
    inputs.input_ids[i, selection[i]] = 0 #valor 0 es el mask!

In [0]:
# inputs.input_ids[1]
# selection[2]



##Quiero que me prediga los valores que fueron padding? creo que noo! revisar eso y ver de pasarlo a unos. trabajar con el len

We can see the values *103* have been assigned in the same positions as we found *True* values in the `mask_arr` tensor.

The `inputs` tensors are now ready, and can we can begin setting them up to be fed into our model during training. We create a PyTorch dataset from our data.

In [0]:
class MeditationsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings
    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
    def __len__(self):
        return len(self.encodings.input_ids)

Initialize our data using the `MeditationsDataset` class.

In [0]:
dataset = MeditationsDataset(inputs)

And initialize the dataloader, which we'll be using to load our data into the model during training.

In [0]:
loader = torch.utils.data.DataLoader(dataset, batch_size=16, shuffle=True)

Now we can move onto setting up the training loop. First we setup GPU/CPU usage.

In [0]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# and move our model over to the selected device
model.to(device)

Out[21]: BertForMaskedLM(
 (bert): BertModel(
 (embeddings): BertEmbeddings(
 (word_embeddings): Embedding(31002, 768, padding_idx=0)
 (position_embeddings): Embedding(512, 768)
 (token_type_embeddings): Embedding(2, 768)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 (encoder): BertEncoder(
 (layer): ModuleList(
 (0): BertLayer(
 (attention): BertAttention(
 (self): BertSelfAttention(
 (query): Linear(in_features=768, out_features=768, bias=True)
 (key): Linear(in_features=768, out_features=768, bias=True)
 (value): Linear(in_features=768, out_features=768, bias=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 (output): BertSelfOutput(
 (dense): Linear(in_features=768, out_features=768, bias=True)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 )
 (intermediate): BertIntermediate(
 (dense): Linear(in_features=768, out_features=3072, bias=True)
 )
 (output): BertOutput(
 (dense): Linear(in_features=3072, out_features=768, bias=True)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 )
 (1): BertLayer(
 (attention): BertAttention(
 (self): BertSelfAttention(
 (query): Linear(in_features=768, out_features=768, bias=True)
 (key): Linear(in_features=768, out_features=768, bias=True)
 (value): Linear(in_features=768, out_features=768, bias=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 (output): BertSelfOutput(
 (dense): Linear(in_features=768, out_features=768, bias=True)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 )
 (intermediate): BertIntermediate(
 (dense): Linear(in_features=768, out_features=3072, bias=True)
 )
 (output): BertOutput(
 (dense): Linear(in_features=3072, out_features=768, bias=True)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 )
 (2): BertLayer(
 (attention): BertAttention(
 (self): BertSelfAttention(
 (query): Linear(in_features=768, out_features=768, bias=True)
 (key): Linear(in_features=768, out_features=768, bias=True)
 (value): Linear(in_features=768, out_features=768, bias=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 (output): BertSelfOutput(
 (dense): Linear(in_features=768, out_features=768, bias=True)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 )
 (intermediate): BertIntermediate(
 (dense): Linear(in_features=768, out_features=3072, bias=True)
 )
 (output): BertOutput(
 (dense): Linear(in_features=3072, out_features=768, bias=True)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 )
 (3): BertLayer(
 (attention): BertAttention(
 (self): BertSelfAttention(
 (query): Linear(in_features=768, out_features=768, bias=True)
 (key): Linear(in_features=768, out_features=768, bias=True)
 (value): Linear(in_features=768, out_features=768, bias=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 (output): BertSelfOutput(
 (dense): Linear(in_features=768, out_features=768, bias=True)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 )
 (intermediate): BertIntermediate(
 (dense): Linear(in_features=768, out_features=3072, bias=True)
 )
 (output): BertOutput(
 (dense): Linear(in_features=3072, out_features=768, bias=True)
 (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 )
 (4): BertLayer(
 (attention): BertAttention(
 (self): BertSelfAttention(
 (query): Linear(in_features=768, out_features=768, bias=True)
 (key): Linear(in_features=768, out_features=768, bias=True)
 (value): Linear(in_features=768, out_features=768, bias=True)
 (dropout): Dropout(p=0.1, inplace=False)
 )
 (output): BertSelfOutput(
 (dense): Linear(

Activate the training mode of our model, and initialize our optimizer (Adam with weighted decay - reduces chance of overfitting).

In [0]:
from transformers import AdamW

# activate training mode
model.train()
# initialize optimizer
optim = AdamW(model.parameters(), lr=5e-5)

Now we can move onto the training loop, we'll train for two epochs (change `epochs` to modify this).

In [0]:
from tqdm import tqdm  # for our progress bar

epochs = 1

for epoch in range(epochs):
    # setup loop with TQDM and dataloader
    loop = tqdm(loader, leave=True)
    for batch in loop:
        # initialize calculated gradients (from prev step)
        optim.zero_grad()
        # pull all tensor batches required for training
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        # process
        outputs = model(input_ids, attention_mask=attention_mask,
                        labels=labels)
        # extract loss
        loss = outputs.loss
        # calculate loss for every parameter that needs grad update
        loss.backward()
        # update parameters
        optim.step()
        # print relevant info to progress bar
        loop.set_description(f'Epoch {epoch}')
        loop.set_postfix(loss=loss.item())

0%| | 0/42 [00:00<?, ?it/s]<command-4107547303087250>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
 return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
Epoch 0: 0%| | 0/42 [00:15<?, ?it/s]Epoch 0: 0%| | 0/42 [00:15<?, ?it/s, loss=17.5]Epoch 0: 2%|▏ | 1/42 [00:15<10:47, 15.80s/it, loss=17.5]Epoch 0: 2%|▏ | 1/42 [00:29<10:47, 15.80s/it, loss=17.5]Epoch 0: 2%|▏ | 1/42 [00:29<10:47, 15.80s/it, loss=12] Epoch 0: 5%|▍ | 2/42 [00:29<09:31, 14.28s/it, loss=12]Epoch 0: 5%|▍ | 2/42 [00:40<09:31, 14.28s/it, loss=12]Epoch 0: 5%|▍ | 2/42 [00:40<09:31, 14.28s/it, loss=7.52]Epoch 0: 7%|▋ | 3/42 [00:40<08:33, 13.16s/it, loss=7.52]Epoch 0: 7%|▋ | 3/42 [00:52<08:33, 13.16s/it, loss=7.52]Epoch 0: 7%|▋ | 3/42 [00:52<08:33, 13.16s/it, loss=2.81]Epoch 0: 10%|▉ | 4/42 [00:52<07:55, 12.52s/it, loss=2.81]Epoch 0: 10%|▉ | 4/42 [01:04<07:55, 12.52s/it, loss=2.81]Epoch 0: 10%|▉ | 4/42 [01:04<07:55, 12.52s/it, loss=0.635]Epoch 0: 12%|█▏ | 5/42 [01:04<07:33, 12.27s/it, loss=0.635]Epoch 0: 12%|█▏ | 5/42 [01:16<07:33, 12.27s/it, loss=0.635]Epoch 0: 12%|█▏ | 5/42 [01:16<07:33, 12.27s/it, loss=0.698]Epoch 0: 14%|█▍ | 6/42 [01:16<07:24, 12.35s/it, loss=0.698]Epoch 0: 14%|█▍ | 6/42 [01:28<07:24, 12.35s/it, loss=0.698]Epoch 0: 14%|█▍ | 6/42 [01:28<07:24, 12.35s/it, loss=1.23] Epoch 0: 17%|█▋ | 7/42 [01:28<07:09, 12.26s/it, loss=1.23]Epoch 0: 17%|█▋ | 7/42 [01:40<07:09, 12.26s/it, loss=1.23]Epoch 0: 17%|█▋ | 7/42 [01:40<07:09, 12.26s/it, loss=1.45]Epoch 0: 19%|█▉ | 8/42 [01:40<06:51, 12.11s/it, loss=1.45]Epoch 0: 19%|█▉ | 8/42 [01:52<06:51, 12.11s/it, loss=1.45]Epoch 0: 19%|█▉ | 8/42 [01:52<06:51, 12.11s/it, loss=2.23]Epoch 0: 21%|██▏ | 9/42 [01:52<06:36, 12.00s/it, loss=2.23]Epoch 0: 21%|██▏ | 9/42 [02:06<06:36, 12.00s/it, loss=2.23]Epoch 0: 21%|██▏ | 9/42 [02:06<06:36, 12.00s/it, loss=1.13]Epoch 0: 24%|██▍ | 10/42 [02:06<06:42, 12.57s/it, loss=1.13]Epoch 0: 24%|██▍ | 10/42 [02:23<06:42, 12.57s/it, loss=1.13]Epoch 0: 24%|██▍ | 10/42 [02:23<06:42, 12.57s/it, loss=0.924]Epoch 0: 26%|██▌ | 11/42 [02:23<07:17, 14.11s/it, loss=0.924]Epoch 0: 26%|██▌ | 11/42 [02:40<07:17, 14.11s/it, loss=0.924]Epoch 0: 26%|██▌ | 11/42 [02:40<07:17, 14.11s/it, loss=0.633]Epoch 0: 29%|██▊ | 12/42 [02:40<07:31, 15.04s/it, loss=0.633]Epoch 0: 29%|██▊ | 12/42 [02:56<07:31, 15.04s/it, loss=0.633]Epoch 0: 29%|██▊ | 12/42 [02:56<07:31, 15.04s/it, loss=0.72] Epoch 0: 31%|███ | 13/42 [02:56<07:22, 15.27s/it, loss=0.72]Epoch 0: 31%|███ | 13/42 [03:11<07:22, 15.27s/it, loss=0.72]Epoch 0: 31%|███ | 13/42 [03:11<07:22, 15.27s/it, loss=0.339]Epoch 0: 33%|███▎ | 14/42 [03:11<07:00, 15.02s/it, loss=0.339]Epoch 0: 33%|███▎ | 14/42 [03:22<07:00, 15.02s/it, loss=0.339]Epoch 0: 33%|███▎ | 14/42 [03:22<07:00, 15.02s/it, loss=0.29] Epoch 0: 36%|███▌ | 15/42 [03:22<06:18, 14.03s/it, loss=0.29]Epoch 0: 36%|███▌ | 15/42 [03:34<06:18, 14.03s/it, loss=0.29]Epoch 0: 36%|███▌ | 15/42 [03:34<06:18, 14.03s/it, loss=0.321]Epoch 0: 38%|███▊ | 16/42 [03:34<05:42, 13.16s/it, loss=0.321]Epoch 0: 38%|███▊ | 16/42 [03:45<05:42, 13.16s/it, loss=0.321]Epoch 0: 38%|███▊ | 16/42 [03:45<05:42, 13.16s/it, loss=0.362]Epoch 0: 40%|████ | 17/42 [03:45<05:17, 12.68s/it, loss=0.362]Epoch 0: 40%|████ | 17/42 [03:57<05:17, 12.68s/it, loss=0.362]Epoch 0: 40%|████ | 17/42 [03:57<05:17, 12.68s/it, loss=0.268]Epoch 0: 43%|████▎ | 18/42 [03:57<04:58, 12.43s/it, loss=0.268]Epoch 0: 43%|████▎ | 18/42 [04:09<04:58, 12.43s/it, loss=0.268]Epoch 0: 43%|████▎ | 18/42 [04:09<04:58, 12.43s/it, loss=0.296]Epoch 0: 45%|████▌ | 19/42 [04:09<04:42, 12.27s/it, loss=0.296]Epoch 0: 45%|████▌ | 19/42 [04:21<04:42, 12.27s/it, loss=0.296]Epoch 0: 45%|████▌ | 19/42 [04:21<04:42, 12.27s/it, loss=0.334]Epoch 0: 48%|████▊ | 20/42 [04:21<04:28, 12.21s/it, loss=0.334]Epoch 0: 48%|████▊ | 20/42 [04:33<04:28, 12.21s/it, loss=0.334]E

In [0]:
# Save model
# model.save_pretrained(f'/dbfs/Efecty_VoC/test/BETOV4_upd')
# dbutils.fs.cp(f'mnt/contenedor/Efecty_VoC/NLP Models/BETO/vocab.txt',f'/dbfs/Efecty_VoC/test/BETOV4_upd/vocab.txt')


#otra forma
# name_file=f'/dbfs/Efecty_VoC/BETOmodel/BETO_UPDATEV1_con_textos_colab_red.pkl'
# torch.save(model,name_file)
# dbutils.fs.cp(f'/Efecty_VoC/BETOmodel/Modelo_Tema1_{str(dt.date.today())}.pkl',f'/mnt/contenedor/Efecty_VoC/NLP Models/Modelos/Modelo_Tema1_{str(dt.date.today())}.pkl')

Out[24]: True

And there we go, we've fine-tuned our BERT model using MLM on *Meditations*!